<a href="https://colab.research.google.com/github/johannnamr/Discrepancy-based-inference-using-QMC/blob/main/Inference/G-and-k-distribution/gandk_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# G-and-k distribution

Notebook for the estimation of the model parameters

### Mount Drive

In [ ]:
# mount my drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


The following path has to be adjusted if necessary:

In [ ]:
%run "/content/drive/My Drive/Colab Notebooks/Utils.ipynb"
%run "/content/drive/My Drive/Colab Notebooks/Plot_fcts.ipynb"

     |████████████████████████████████| 1.6MB 5.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 4.0 MB 5.8 MB/s 
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  distutils: /usr/local/include/python3.7/qmcpy
  sysconfig: /usr/include/python3.7m/qmcpy
  distutils: /usr/local/bin
  sysconfig: /usr/bin
  distutils: /usr/local
  syscon

Set path for saving the results:

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/Paper/Inference/'

### Imports

In [ ]:
import numpy as np

### Optimisation

In [ ]:
np.random.seed(11)

Set parameters:

In [ ]:
m = 2**14                                         # number of true data points
n = 2**9                                          # number of simulated data points
theta_star = np.array([3,1,1,-np.log(2)])         # true theta
d = 1                                             # dimensions of data
p = theta_star.shape[0]                           # dimensions of parameter space
kernel = 'gaussian'                               # 'gaussian' or 'imq'
l = 1.5                                            # lengthscale for Gaussian kernel (l=-1 for median heuristic)
c=1                                               # parameter c for IMQ kernel
b=-0.5                                            # parameter beta for IMQ kernel
nu = 0                                            # parameter nu for matern kernel
method_gd = 'SGD'                                 # SGD or NSGD
eta = 0.05                                         # step size
max_it = 10000                                    # number of descent steps
start = np.ones(p)*0.5                            # start values
inv = True                                        # True: use inverse transform, False: use Box-Muller transform
stat_type = 'v'                                   # 'u': U-statistic, 'v': V-statistic

Define QMC sequence:

In [ ]:
if inv:
  qmc = qmcpy.Halton(d)
else:  
  qmc = qmcpy.Halton(d+1)

Sample from the g-and-k distribution:

In [ ]:
if inv:
  y,_ = sample_gandk_inv('MC',m,d,theta_star)
else:
  y,_ = sample_gandk('MC',m,d,theta_star)

Gradient descent:

In [ ]:
if inv:
  theta_mc,loss_mc = optim_inv_nosub('gandk','MC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)
else:
  theta_mc,loss_mc = optim('gandk','MC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)

In [ ]:
if inv:
  theta_qmc,loss_qmc = optim_inv_nosub('gandk','QMC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)
else:
  theta_qmc,loss_qmc = optim('gandk','QMC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)

In [ ]:
if inv:
  theta_rqmc,loss_rqmc = optim_inv_nosub('gandk','RQMC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)
else:
  theta_rqmc,loss_rqmc = optim('gandk','RQMC',method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel)

### Visualise results

In [ ]:
it = 10000

Plot the loss:

In [ ]:
plot_loss(it,d,(12,6),loss_mc,loss_qmc,loss_rqmc,'MC','QMC','RQMC')

Plot the estimates:

In [ ]:
plot_estimates(2,p,d,(12,((p//2)+(p%2))*6),max_it,theta_mc,theta_qmc,theta_rqmc,theta_star,'MC','QMC','RQMC')

Plot the MSE:

In [ ]:
mse_mc,mse_qmc,mse_rqmc = mse(max_it,p,theta_mc,theta_qmc,theta_rqmc,theta_star)

In [ ]:
plot_mse(2,p,d,(18,((p//2)+(p%2))*9),max_it,mse_mc,mse_qmc,mse_rqmc,'MC','QMC','RQMC','gandk')

Save the results:

In [ ]:
np.savez(path+"gandk_mmd2_optim_d=%s_%sstat_k%s_m=%s_eta=%s.npz" %(d,stat_type,kernel,m,eta),loss_mc=loss_mc,loss_qmc=loss_qmc,loss_rqmc=loss_rqmc,theta_mc=theta_mc,theta_qmc=theta_qmc,theta_rqmc=theta_rqmc,mse_mc=mse_mc,mse_qmc=mse_qmc,mse_rqmc=mse_rqmc)